In [5]:
#!pip3 install webdriver_manager

In [6]:
#!pip3 install selenium

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

### Abrindo Página da Hotmart

In [24]:
endereco = 'https://app.hotmart.com/market'

options = Options()
options.add_argument('window-size=900,660')
service = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=service, options=options)
navegador.get(endereco)

### Acessando a conta da Hotmart

In [25]:
cont = 0
while True:
    sleep(1)
    try:
        input_user = navegador.find_element(By.ID, 'username')
        break
    except:
        cont += 1
        if cont > 15:
            break

input_user.send_keys('pedro_h2o_88@hotmail.com')
input_password = navegador.find_element(By.ID, 'password')
input_password.send_keys('15D&novembro')
navegador.find_element(By.NAME, 'submit').click()

### Filtrando os produtos por último clique

In [26]:
cont = 0
while True:
    sleep(1)
    try:
        navegador.find_element(By.XPATH, '//*[@id="vulcano-market"]/div/div/div[1]/div[2]/div[2]/button/div').click()
        break
    except:
        cont += 1
        if cont > 15:
            break

navegador.find_element(By.ID, 'commissioningFilter').click()
input('Apos selecionar aperte enter: ')
navegador.find_element(By.XPATH, '//*[@id="vulcano-market"]/div/div/div[1]/div[3]/hot-card/hot-card-body/div[5]/button[2]').click()

### Capturando a página com Beautiful Soup

In [27]:
cont = 0
while True:
    sleep(1)
    page_content = navegador.page_source
    site = BeautifulSoup(page_content, 'html.parser')    
    try:
        if site.find('div', attrs={'class': 'hot-col-xl-3 hot-col-lg-4 hot-col-md-6 hot-col-sm-12 _py-3'}):
            break
    except:
        cont += 1
        if cont > 60:
            break

produtos = site.find_all('div', attrs={'class': 'hot-col-xl-3 hot-col-lg-4 hot-col-md-6 hot-col-sm-12 _py-3'})
print(produtos)

[<div class="hot-col-xl-3 hot-col-lg-4 hot-col-md-6 hot-col-sm-12 _py-3"><a class="product-detail-link" data-test-id="product-card-link" href="/market/details?producerUcode=4ff91939-a93b-4f43-b690-42a723c3fa90&amp;productUcode=a37d5d27-1072-4ca6-9b0d-0ff5ac0e56da&amp;bookmarked=false&amp;searchId=64e07fc7-0848-4fb0-8c1f-1b20b8435302"><hot-card class="product-card card-container _border-0 _rounded-lg _w-full _mb-3"><hot-card-header class="product-card-header _p-0 _border-0 _rounded-0 _position-relative"><div class="product-image-background _position-relative" style='background-image: url("https://static-media.hotmart.com/CPC9b96G8cyfkDPKKnuTw9ajt3I=/filters:background_color(white)/hotmart/product_pictures/bae4fdf8-11ea-4ae4-924d-8b69084ab3cc/altaintensidade.jpg");'></div><div class="product-image _position-absolute _d-flex _w-full _align-items-center _justify-content-center"><img alt="" class="_h-full" src="https://static-media.hotmart.com/CPC9b96G8cyfkDPKKnuTw9ajt3I=/filters:background

### Capturando informações da página 

In [17]:
list_produtos = []
for produto in produtos:
    try:
        link_produto = 'https://app.hotmart.com/'+(produto.find('a', attrs={'class': 'product-detail-link'})['href'])
    except:
        link_produto = '-'
    try:
        nome_produto = str(produto.find('span', attrs={'class': 'product-name _text-md-2 _text-gray-800'}).string)
    except:
        nome_produto = '-'
    try:
        qtd_avaliacoes = int(produto.find('span', attrs={'class': '_ml-1 _text-1 _text-gray-500 _font-weight _d-none _d-md-inline'}).string.lstrip('(').rstrip(')'))
    except:
        qtd_avaliacoes = '-'
    try:
        temperatura_produto = int(produto.find_all('span', attrs={'class': '_mr-1 _text-1 _text-gray-800'}, limit=2)[1].string.rstrip('°'))
    except:
        temperatura_produto = '-'
    try:
        comissao = float(produto.find('p', attrs={'class': '_mb-0 _text-3 _text-md-4 _text-green _font-weight-light'}).string.lstrip('R$ ').replace(',','.'))
    except:
        comissao = '-'
    try:
        preco = float(produto.find('p', attrs={'class': '_mb-0 _text-1 _text-gray-500'}).string.lstrip('Preço máximo do produto: R$ ').replace(',','.'))
    except:
        preco = '-'

    try:
        if comissao > 10 and temperatura_produto > 20:
            list_produtos.append([nome_produto, link_produto, qtd_avaliacoes, temperatura_produto, comissao, preco])
    except:
        pass

print(list_produtos)

[['Desafio sem medidas ', 'https://app.hotmart.com//market/details?producerUcode=416cd0d0-b1bd-48c3-84b7-3423f585cc10&productUcode=508b5252-c9e8-46cd-9fe3-ea7c38bdad9a&bookmarked=false&searchId=7b9cea16-61f0-4f5d-a0d3-ba94f90251c8', 0, 21, 28.4, 63.59], ['15 SEGREGOS PARA EMAGRECER', 'https://app.hotmart.com//market/details?producerUcode=f47784e8-2f97-492e-b109-cc4b40896daa&productUcode=bc1bb23a-8767-489a-8248-73c1e0e9529f&bookmarked=false&searchId=7b9cea16-61f0-4f5d-a0d3-ba94f90251c8', 0, 21, 10.46, 37.99], ['Guia Completo para Reduzir as Estrias e Melhorar a Aparência da Pele ', 'https://app.hotmart.com//market/details?producerUcode=02cf8fa9-5c30-4473-b6a6-2a6f45af1457&productUcode=4aa6a7a1-56ba-4d63-809f-ba861fc6e4a6&bookmarked=false&searchId=7b9cea16-61f0-4f5d-a0d3-ba94f90251c8', 0, 21, 17.76, 39.97], ['Receitas Low Carb para perder peso', 'https://app.hotmart.com//market/details?producerUcode=b842f809-8b22-48cc-9261-99a96a4e89b0&productUcode=8d901b60-97fd-4660-9b10-3b1239ef1257&bo

In [36]:
print(site.find_all('svg', attrs={'data-icon': 'chevron-right'}))

[<svg aria-hidden="true" class="svg-inline--fa fa-chevron-right fa-w-8 _p-0 _text-3" data-icon="chevron-right" data-prefix="far" focusable="false" role="img" viewbox="0 0 256 512" xmlns="http://www.w3.org/2000/svg"><path d="M24.707 38.101L4.908 57.899c-4.686 4.686-4.686 12.284 0 16.971L185.607 256 4.908 437.13c-4.686 4.686-4.686 12.284 0 16.971L24.707 473.9c4.686 4.686 12.284 4.686 16.971 0l209.414-209.414c4.686-4.686 4.686-12.284 0-16.971L41.678 38.101c-4.687-4.687-12.285-4.687-16.971 0z" fill="currentColor"></path></svg>]


In [5]:
# Abrindo a Hotmart
endereco = 'https://app.hotmart.com/market'

options = Options()
options.add_argument('window-size=900,660')
service = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=service, options=options)
navegador.get(endereco)

list_produtos = []

# Acessando a conta hotmart
cont = 0
while True:
    sleep(1)
    try:
        input_user = navegador.find_element(By.ID, 'username')
        break
    except:
        cont += 1
        if cont > 15:
            break

input_user.send_keys('seu-email@email.com')
input_password = navegador.find_element(By.ID, 'password')
input_password.send_keys('sua-senha')
navegador.find_element(By.NAME, 'submit').click()

# Filtrando produtos por comissão por último clique
cont = 0
while True:
    sleep(1)
    try:
        navegador.find_element(By.XPATH, '//*[@id="vulcano-market"]/div/div/div[1]/div[2]/div[2]/button/div').click()
        break
    except:
        cont += 1
        if cont > 15:
            break

navegador.find_element(By.ID, 'commissioningFilter').click()
input('Apos selecionar aperte enter: ')
navegador.find_element(By.XPATH, '//*[@id="vulcano-market"]/div/div/div[1]/div[3]/hot-card/hot-card-body/div[5]/button[2]').click()

# Capturando a página com Beautiful Soap
tem_botao = True
passagem = 0
while True:
    cont = 0
    while True:
        sleep(1)
        page_content = navegador.page_source
        site = BeautifulSoup(page_content, 'html.parser')    
        try:
            if site.find('div', attrs={'class': 'hot-col-xl-3 hot-col-lg-4 hot-col-md-6 hot-col-sm-12 _py-3'}):
                break
        except:
            cont += 1
            if cont > 60:
                break

    produtos = site.find_all('div', attrs={'class': 'hot-col-xl-3 hot-col-lg-4 hot-col-md-6 hot-col-sm-12 _py-3'})
    #print(produtos)

    # Capturando informações da página    
    for produto in produtos:
        try:
            link_produto = 'https://app.hotmart.com'+(produto.find('a', attrs={'class': 'product-detail-link'})['href'])
        except:
            link_produto = '-'
        try:
            nome_produto = str(produto.find('span', attrs={'class': 'product-name _text-md-2 _text-gray-800'}).string)
        except:
            nome_produto = '-'
        try:
            qtd_avaliacoes = int(produto.find('span', attrs={'class': '_ml-1 _text-1 _text-gray-500 _font-weight _d-none _d-md-inline'}).string.lstrip('(').rstrip(')'))
        except:
            qtd_avaliacoes = '-'
        try:
            temperatura_produto = int(produto.find_all('span', attrs={'class': '_mr-1 _text-1 _text-gray-800'}, limit=2)[1].string.rstrip('°'))
        except:
            temperatura_produto = '-'
        try:
            comissao = float(produto.find('p', attrs={'class': '_mb-0 _text-3 _text-md-4 _text-green _font-weight-light'}).string.lstrip('R$ ').replace(',','.'))
        except:
            comissao = '-'
        try:
            preco = float(produto.find('p', attrs={'class': '_mb-0 _text-1 _text-gray-500'}).string.lstrip('Preço máximo do produto: R$ ').replace(',','.'))
        except:
            preco = '-'

        try:
            if comissao > 89.99 and temperatura_produto > 21 and temperatura_produto < 81 and qtd_avaliacoes > 1:
                list_produtos.append([nome_produto, link_produto, qtd_avaliacoes, temperatura_produto, comissao, preco])
        except:
            pass

    passagem += 1
    
    paginas = site.find_all('hot-pagination-item')
    for k, i in enumerate(paginas):
        temporario = str(i)
        if 'hot-pagination__item--active' in temporario:
            proxima_pagina = k+2
            #print(f'proxima pagina -> {proxima_pagina}')

    if navegador.find_element(By.XPATH, f'//*[@id="vulcano-market"]/div/div[4]/hot-pagination/hot-pagination-item[{proxima_pagina}]'):
        navegador.find_element(By.XPATH, f'//*[@id="vulcano-market"]/div/div[4]/hot-pagination/hot-pagination-item[{proxima_pagina}]').click()
    else:
        break

    if passagem > 300:
        break

data = list_produtos
df = pd.DataFrame(data, columns=['NomeProduto', 'EnderecoWeb','Avaliacoes','Temperatura','Comissao','Preco'])
df
    

,NomeProduto,EnderecoWeb,Avaliacoes,Temperatura,Comissao,Preco
0,Curso Viagem com Milhas,https://app.hotmart.com/market/details?produce...,5,39,116.81,-
1,Leitura técnica automotiva,https://app.hotmart.com/market/details?produce...,4,22,133.55,297.0
2,NOVO Curso Como Tocar Cavaquinho 3.0 - Plano V...,https://app.hotmart.com/market/details?produce...,28,23,223.65,497.0
3,Metodo Lancha,https://app.hotmart.com/market/details?produce...,6,24,342.33,-
4,Curso Meu Tricotin,https://app.hotmart.com/market/details?produce...,108,73,178.60,397.0
5,Curso Mandaleira Completa,https://app.hotmart.com/market/details?produce...,67,27,179.56,997.0
6,Monografis - Orientador TCC,https://app.hotmart.com/market/details?produce...,881,48,225.89,497.0
7,Programa Especialista em Injeções Programáveis!,https://app.hotmart.com/market/details?produce...,49,34,534.71,-
8,Curso Viagem com Milhas,https://app.hotmart.com/market/details?produce...,5,39,116.81,-
9,SACHI • ESCOLA DE SABOARIA NATURAL,https://app.hotmart.com/market/details?produce...,9,23,396.33,-


In [6]:
df.to_csv("lista_produtos.csv", encoding = 'utf-8', index = False)